In [1]:
import datetime
import pandas as pd

start = datetime.datetime.strptime("01-01-2018", "%d-%m-%Y")
end = datetime.datetime.strptime("26-01-2019", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days = x) for x in range(0, (end - start).days)]

dates_list = []
for date in date_generated:
    txt = str(str(date.day) + '.' + str(date.month) + '.' + str(date.year))
    dates_list.append(txt)

ndf = pd.DataFrame()  # create empty ndf
for i in range(0, len(dates_list)):
    allURL = 'https://www.uzse.uz/trade_results?date=' + dates_list[i] + '&locale=en&mkt_id=ALL&page=%d'

    for k in range(1, 100):
        url = allURL % k
        if pd.read_html(url)[0].empty:
            break
        else:
            chunk = pd.read_html(url)[0]
            chunk['Date'] = dates_list[i] # Date is positioned at last position, let's fix that
            cols = chunk.columns.tolist() # get a list of all the columns
            cols = cols[-1:] + cols[:-1] # rearrange the columns, move the last element (Date) to the first position
            chunk = chunk[cols] # reorder the dataframe
            ndf = pd.concat([ndf, chunk])

print(ndf)

         Date      Time                 ISIN                       Isu Name  \
0    4.1.2018  15:02:00   UZ7035530003  TNBN                <Turonbank> ATB   
0    5.1.2018  13:18:29   UZ7055790008  AABK       <Asia Alliance Bank> ATB   
1    5.1.2018  13:17:40   UZ7055790008  AABK       <Asia Alliance Bank> ATB   
2    5.1.2018  13:16:47   UZ7055790008  AABK       <Asia Alliance Bank> ATB   
0   11.1.2018  12:36:36   UZ705533K016  UNGC  <O'zneftgazqazibchiqarish> AJ   
0   12.1.2018  11:48:12   UZ7039920002  IPKY              AITB <Ipak Yo'li>   
0   15.1.2018  14:29:46   UZ7011340005  HMKB               <Hamkorbank> ATB   
1   15.1.2018  14:27:03   UZ7011340005  HMKB               <Hamkorbank> ATB   
2   15.1.2018  14:23:39   UZ7011340005  HMKB               <Hamkorbank> ATB   
3   15.1.2018  14:19:20   UZ7011340005  HMKB               <Hamkorbank> ATB   
0   16.1.2018  15:02:00   UZ7018790004  KKBN    ATB <Qishloq qurilish bank>   
0   17.1.2018  15:02:00   UZ7045320007  ALSM        

Let's see what are the names of the **Columns** in the data frame:

In [2]:
ndf.columnsns

Index(['Date', 'Time', 'ISIN', 'Isu Name', 'Тип ценной бумаги', 'Market',
       'BRD ID', 'Trade Price', 'Trading Volumn', 'Trading Value'],
      dtype='object')

Before moving forward we should get rid off the column with funny letters. Rename the column 5:

In [7]:
ndf.columns.values[4] = 'Security Type'
ndf.columns

Index(['Date', 'Time', 'ISIN', 'Isu Name', 'Security Type', 'Market', 'BRD ID',
       'Trade Price', 'Trading Volumn', 'Trading Value'],
      dtype='object')

Now using the column **"Market"**, we can see how many transactions or trades completed for each unique type of the Market:

In [34]:
ndf.Market.value_counts()

STK    15051
BND        3
Name: Market, dtype: int64

From the results above, it appears that there were only three trades related to Bonds, while the total number of trades with Stocks is equal to 15051. Let's look at the trades with Bonds:

In [37]:
bdf = ndf.loc[ndf['Market'] == 'BND']
#bdf['Date'] = bdf['Date'].str.replace('.','-')
#bdf['Date'] = pd.to_datetime(bdf['Date'], format='%d-%m-%Y')
bdfData = bdf.to_dict(orient='list')
bdfData

{'BRD_ID': ['G1', 'G1', 'G1'],
 'Date': [Timestamp('2018-07-16 00:00:00'),
  Timestamp('2018-08-15 00:00:00'),
  Timestamp('2018-11-19 00:00:00')],
 'ISIN': ['UZ6037561032  SQB3', 'UZ6037561032  SQB3', 'UZ6047441VA5  KPB2'],
 'Isu_Name': ["<O''zsanoatqurilishbank> ATB",
  "<O''zsanoatqurilishbank> ATB",
  '<Kapitalbank> ATB'],
 'Market': ['BND', 'BND', 'BND'],
 'Security_Type': ['Облигации', 'Облигации', 'Облигации'],
 'Time': ['12:47:17', '14:28:11', '12:21:26'],
 'Trade_Price': [103000.0, 103000.0, 1000000.0],
 'Trading_Value': ['UZS  103,000', 'UZS  7,107,000', 'UZS  230,000,000'],
 'Trading_Volumn': [1, 69, 230]}

Let's now convert the dates to proper date format:

In [11]:
ddf = ndf
ddf['Date'] = ddf['Date'].str.replace('.','-')
ddf['Date'] = pd.to_datetime(ddf['Date'], format='%d-%m-%Y')
ddf.head()

,Date,Time,ISIN,Isu Name,Security Type,Market,BRD ID,Trade Price,Trading Volumn,Trading Value
0,2018-01-04,15:02:00,UZ7035530003 TNBN,<Turonbank> ATB,Простыеакции,STK,G1,1700.0,20,"UZS 34,000"
0,2018-01-05,13:18:29,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,"UZS 1,693,237,000"
1,2018-01-05,13:17:40,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,"UZS 1,693,237,000"
2,2018-01-05,13:16:47,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,"UZS 1,693,237,000"
0,2018-01-11,12:36:36,UZ705533K016 UNGC,<O'zneftgazqazibchiqarish> AJ,Прив.акции,STK,G1,1000.0,58,"UZS 58,000"


See what Security Type column has in it. But before that, we need to replace **' '** with **'_'** in columns names to make pandas functionality work better:

In [17]:
ddf.columns = ddf.columns.str.replace(' ', '_')
ddf.Security_Type.value_counts()

Простыеакции    13571
Прив.акции       1480
Облигации           3
Name: Security_Type, dtype: int64

Nowe we replace those funny words to proper ones:

In [20]:
replace_values = {'Простыеакции' : 'CommonStock', 'Прив.акции' : 'PrefStock', 'Облигации' : 'Bond'}                                                                                          
ddf = ddf.replace({"Security_Type": replace_values}) 
ddf.Security_Type.value_counts()

CommonStock    13571
PrefStock       1480
Bond               3
Name: Security_Type, dtype: int64

Let's see how many Issuers we have traded since September, 2016:

In [25]:
ddf.head()

,Date,Time,ISIN,Isu_Name,Security_Type,Market,BRD_ID,Trade_Price,Trading_Volumn,Trading_Value
0,2018-01-04,15:02:00,UZ7035530003 TNBN,<Turonbank> ATB,CommonStock,STK,G1,1700.0,20,"UZS 34,000"
0,2018-01-05,13:18:29,UZ7055790008 AABK,<Asia Alliance Bank> ATB,CommonStock,STK,G1,260.0,6512450,"UZS 1,693,237,000"
1,2018-01-05,13:17:40,UZ7055790008 AABK,<Asia Alliance Bank> ATB,CommonStock,STK,G1,260.0,6512450,"UZS 1,693,237,000"
2,2018-01-05,13:16:47,UZ7055790008 AABK,<Asia Alliance Bank> ATB,CommonStock,STK,G1,260.0,6512450,"UZS 1,693,237,000"
0,2018-01-11,12:36:36,UZ705533K016 UNGC,<O'zneftgazqazibchiqarish> AJ,PrefStock,STK,G1,1000.0,58,"UZS 58,000"


Top 10 transactions look like as:

In [32]:
Trade_by_Issuers = ddf['Isu_Name'].value_counts()\
                 .to_frame('Transactions').rename_axis('Issuers')\
                 .reset_index()
Trade_by_Issuers.head(10)

,Issuers,Transactions
0,<Kvarts> AJ,3576
1,<Qo'qon mexanika zavodi> AJ,3259
2,<Qizilqumsement> AJ,1803
3,<O'zsanoatqurilishbank> ATB,1140
4,<Hamkorbank> ATB,993
5,<Andijon tajriba-sinov zavodi> AJ,442
6,<Ipoteka-bank> ATIB,407
7,<O'zbekko'mir> AJ,399
8,<Alskom> SK AJ,202
9,<Sarbon-Neftegaz> AJ,196


In order to see the characteristics of each column (date, number, string, etc.) we do:

In [18]:
ddf.dtypes

Date              datetime64[ns]
Time                      object
ISIN                      object
Isu_Name                  object
Security_Type             object
Market                    object
BRD_ID                    object
Trade_Price              float64
Trading_Volumn             int64
Trading_Value             object
dtype: object

In [38]:
ndf.head()

,Date,Time,ISIN,Isu_Name,Security_Type,Market,BRD_ID,Trade_Price,Trading_Volumn,Trading_Value
0,2018-01-04,15:02:00,UZ7035530003 TNBN,<Turonbank> ATB,Простыеакции,STK,G1,1700.0,20,"UZS 34,000"
0,2018-01-05,13:18:29,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,"UZS 1,693,237,000"
1,2018-01-05,13:17:40,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,"UZS 1,693,237,000"
2,2018-01-05,13:16:47,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,"UZS 1,693,237,000"
0,2018-01-11,12:36:36,UZ705533K016 UNGC,<O'zneftgazqazibchiqarish> AJ,Прив.акции,STK,G1,1000.0,58,"UZS 58,000"


The column 'Trading_Value' is not a number, thus we need to multiply the price by quantity to get the total value of a transaction:

In [49]:
import numpy as np
ndf['Trade_Price'] = ndf['Trade_Price'].apply(np.float64)
ndf['Trading_Value'] = ndf.Trade_Price * ndf.Trading_Volumn
ndf.head()

,Date,Time,ISIN,Isu_Name,Security_Type,Market,BRD_ID,Trade_Price,Trading_Volumn,Trading_Value
0,2018-01-04,15:02:00,UZ7035530003 TNBN,<Turonbank> ATB,Простыеакции,STK,G1,1700.0,20,3.400000e+04
0,2018-01-05,13:18:29,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,1.693237e+09
1,2018-01-05,13:17:40,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,1.693237e+09
2,2018-01-05,13:16:47,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,1.693237e+09
0,2018-01-11,12:36:36,UZ705533K016 UNGC,<O'zneftgazqazibchiqarish> AJ,Прив.акции,STK,G1,1000.0,58,5.800000e+04


Let's try to format the number and make them look nice:

In [50]:
ndf = ndf.round(2)
ndf.head()

,Date,Time,ISIN,Isu_Name,Security_Type,Market,BRD_ID,Trade_Price,Trading_Volumn,Trading_Value
0,2018-01-04,15:02:00,UZ7035530003 TNBN,<Turonbank> ATB,Простыеакции,STK,G1,1700.0,20,3.400000e+04
0,2018-01-05,13:18:29,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,1.693237e+09
1,2018-01-05,13:17:40,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,1.693237e+09
2,2018-01-05,13:16:47,UZ7055790008 AABK,<Asia Alliance Bank> ATB,Простыеакции,STK,G1,260.0,6512450,1.693237e+09
0,2018-01-11,12:36:36,UZ705533K016 UNGC,<O'zneftgazqazibchiqarish> AJ,Прив.акции,STK,G1,1000.0,58,5.800000e+04


In [51]:
ndf.dtypes

Date              datetime64[ns]
Time                      object
ISIN                      object
Isu_Name                  object
Security_Type             object
Market                    object
BRD_ID                    object
Trade_Price              float64
Trading_Volumn             int64
Trading_Value            float64
dtype: object

In [52]:
minim = ndf['Trade_Price'].min()
minim

0.0